In [1]:
import pandas as pd
import os
from datetime import date, timedelta
import datetime
import numpy as np
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from statsmodels.tsa.statespace.sarimax import SARIMAX
# grid search sarima hyperparameters for monthly mean temp dataset
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from sklearn.metrics import mean_squared_error
import altair as alt
ROOT_DIR = os.path.realpath(os.path.join(os.getcwd(), '..'))


In [5]:
pkl_loc = os.path.join(ROOT_DIR, 'data', 'data','weatherAA.pkl')
df_org=pd.read_pickle(pkl_loc)


In [10]:
pkl_loc = os.path.join(ROOT_DIR, 'data','badAAcleaned.pkl')
df_bad=pd.read_pickle(pkl_loc)
df_bad.drop(columns=['index'], inplace=True)
df_bad.set_index('time',inplace=True)

In [13]:
pkl_loc = os.path.join(ROOT_DIR, 'data','AAcleaned.pkl')
df_cln=pd.read_pickle(pkl_loc)


In [21]:
time_range = pd.date_range('2006-04-08 00:00:00','2006-04-11 23:00:00', freq='H') #'2022-10-15 23:00:00', freq='H') ##creating time range
time_range=pd.DataFrame(time_range, columns=['time'])
time_range.set_index('time',inplace=True)


### Cleaning - issues with Polynomial

In [59]:
df_org_clean=time_range.merge(df_org.loc['2006-04-08 00:00:00':'2006-04-11 23:00:00']['temp'],how='left',left_index=True,right_index=True)
df_org_clean['display']='Original'
org = alt.Chart(df_org_clean.reset_index(), height=400, width=800).mark_circle(color='blue').encode(
alt.X('time'),
alt.Y('temp'),
color='Original:N'

)

poly=alt.Chart(df_bad.loc['2006-04-09 04:00:00':'2006-04-10 16:00:00'].reset_index()).mark_circle(color='red',height=150, width=150).encode(
x=alt.X('time'),
y=alt.Y('temp'),
color='temp:T'
)

arima=alt.Chart(df_cln.loc['2006-04-09 04:00:00':'2006-04-10 16:00:00'].reset_index()).mark_circle(color='green',height=150, width=150).encode(
x=alt.X('time'),
y=alt.Y('temp')
)

#stacked=(org+poly+arima
#).properties(
#    title
org

alt.Chart(...)

In [25]:
print(df_org_clean.to_string())

                     temp
time                     
2006-04-08 00:00:00   2.8
2006-04-08 01:00:00   2.2
2006-04-08 02:00:00   2.2
2006-04-08 03:00:00   2.2
2006-04-08 04:00:00   1.7
2006-04-08 05:00:00   1.1
2006-04-08 06:00:00   0.6
2006-04-08 07:00:00   0.6
2006-04-08 08:00:00  -0.6
2006-04-08 09:00:00  -0.6
2006-04-08 10:00:00  -1.1
2006-04-08 11:00:00  -1.7
2006-04-08 12:00:00  -1.1
2006-04-08 13:00:00  -0.6
2006-04-08 14:00:00   1.1
2006-04-08 15:00:00   1.7
2006-04-08 16:00:00   2.8
2006-04-08 17:00:00   3.9
2006-04-08 18:00:00   5.0
2006-04-08 19:00:00   6.1
2006-04-08 20:00:00   6.7
2006-04-08 21:00:00   6.1
2006-04-08 22:00:00   6.7
2006-04-08 23:00:00   6.1
2006-04-09 00:00:00   4.4
2006-04-09 01:00:00   3.3
2006-04-09 02:00:00   0.0
2006-04-09 03:00:00  -2.2
2006-04-09 04:00:00   NaN
2006-04-09 05:00:00   NaN
2006-04-09 06:00:00   NaN
2006-04-09 07:00:00   NaN
2006-04-09 08:00:00   NaN
2006-04-09 09:00:00   NaN
2006-04-09 10:00:00   NaN
2006-04-09 11:00:00   NaN
2006-04-09 1